# Higgs Boson Analysis

**Note:** The following code uses mocked data. Please replace the mocked data with your actual `.root` files.

In [ ]:
import uproot
import awkward as ak
import numpy as np
import matplotlib.pyplot as plt

# Create a dummy ROOT file for demonstration
with uproot.recreate("../data/mock_data.root") as file:
    file["events"] = {
        "Photon_pt": np.random.uniform(20, 100, (1000, 4)),
        "Photon_eta": np.random.uniform(-2.5, 2.5, (1000, 4)),
        "Photon_phi": np.random.uniform(-np.pi, np.pi, (1000, 4)),
        "Photon_mass": np.zeros((1000, 4)),
        "nPhoton": np.random.randint(0, 5, 1000)
    }

In [ ]:
# Load the data from the ROOT file
file = uproot.open("../data/mock_data.root")
tree = file["events"]

# Inspect the branches
print("Branches:", tree.keys())

In [ ]:
# Extract the diphoton events
events = tree.arrays(["Photon_pt", "Photon_eta", "Photon_phi", "Photon_mass", "nPhoton"])
diphoton_events = events[events.nPhoton == 2]

# Calculate the invariant mass
px = diphoton_events.Photon_pt * np.cos(diphoton_events.Photon_phi)
py = diphoton_events.Photon_pt * np.sin(diphoton_events.Photon_phi)
pz = diphoton_events.Photon_pt * np.sinh(diphoton_events.Photon_eta)
E = np.sqrt(px**2 + py**2 + pz**2 + diphoton_events.Photon_mass**2)

p4 = ak.zip({
    "px": px,
    "py": py,
    "pz": pz,
    "E": E
}, with_name="LorentzVector")

diphoton = p4[:, 0] + p4[:, 1]
invariant_mass = diphoton.mass

In [ ]:
# Plot the invariant mass distribution
plt.figure(figsize=(10, 6))
plt.hist(invariant_mass, bins=100, range=(80, 180), histtype="step", label="Invariant Mass")
plt.xlabel("Invariant Mass [GeV]")
plt.ylabel("Events")
plt.title("Diphoton Invariant Mass")
plt.legend()
plt.grid(True)
plt.show()